In [1]:
!pip install matplotlib
!pip install seaborn
!pip install dotenv
!pip install sqlalchemy
!pip install pandas


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 23.1 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.7 MB/s eta 0:00:00
Using cached kiwisolver-1.4.8-cp312-cp312-macosx_11_0_arm64.whl (65 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]


In [3]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd

# Load credentials from .env
load_dotenv()

# Database connection info
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

In [4]:
pd.set_option('display.max_rows', None)

# ======================================================
# 📊 Indeed Jobs - Descriptive Query
# ======================================================

# Business Question:
# What are the most common job titles on Indeed and their average listed salaries?

sql_query = '''
WITH cleaned_salaries AS (
    SELECT 
        job_title,
        CAST(job_salary AS INTEGER) AS salary
    FROM indeed_jobs
    WHERE job_salary ~ '^\d{5,6}$'
),
job_summary AS (
    SELECT 
        job_title,
        COUNT(*) AS job_count,
        AVG(salary) AS avg_salary
    FROM cleaned_salaries
    GROUP BY job_title
)
SELECT 
    job_title,
    job_count,
    ROUND(avg_salary) AS avg_salary
FROM job_summary
ORDER BY job_count DESC
LIMIT 10;
'''

indeed_jobs_summary_df = pd.read_sql(sql_query, con=engine)


<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
/var/folders/s4/h0k4q7kx7s52yzr_bd9cvj140000gn/T/ipykernel_19095/3852868311.py:10: SyntaxWarning: invalid escape sequence '\d'
  sql_query = '''


In [5]:

# Insight:
# Job titles with the highest frequency often have highly variable salary data.

# Recommendation:
# Normalize salary data further or target roles with consistent salary representation.

# Prediction:
# Job roles with frequent listings may face increasing competition and wage pressure.


# ======================================================
# 🕵️ Indeed Jobs - Diagnostic Query
# ======================================================

# Business Question:
# Which companies most frequently post for the top job titles on Indeed?

sql_query = '''
WITH parsed_companies AS (
    SELECT 
        job_title,
        SPLIT_PART(company_name_location, 'Remote', 1) AS company_name
    FROM indeed_jobs
),
ranked_companies AS (
    SELECT 
        job_title,
        TRIM(company_name) AS company_name,
        COUNT(*) AS count,
        RANK() OVER (PARTITION BY job_title ORDER BY COUNT(*) DESC) AS rank
    FROM parsed_companies
    GROUP BY job_title, company_name
)
SELECT 
    job_title,
    company_name,
    count
FROM ranked_companies
WHERE rank = 1
ORDER BY count DESC;
'''

indeed_top_companies_df = pd.read_sql(sql_query, con=engine)


# Insight:
# The most frequent job titles tend to be associated with specific companies repeatedly.

# Recommendation:
# Consider targeting these employers in job scraping or analysis to identify hiring trends.

# Prediction:
# These companies may reflect strong growth or high employee turnover in key roles.


In [ ]:
# 📘 Remote Jobs + Indeed Jobs Analysis Notebook

pd.set_option('display.max_rows', None)

# ======================================================
# 📊 Remote Jobs - Descriptive Query
# ======================================================

# Business Question:
# What are the average salary ranges grouped by job level?

sql_query = '''
WITH salary_data AS (
    SELECT 
        job_level,
        annual_salary_min,
        annual_salary_max
    FROM remote_jobs
    WHERE annual_salary_min IS NOT NULL AND annual_salary_max IS NOT NULL
)
SELECT 
    job_level,
    COUNT(*) AS job_count,
    ROUND(AVG(annual_salary_min)) AS avg_salary_min,
    ROUND(AVG(annual_salary_max)) AS avg_salary_max
FROM salary_data
GROUP BY job_level
ORDER BY avg_salary_max DESC;
'''

remote_salary_df = pd.read_sql(sql_query, con=engine)


# Insight:
# Senior level jobs tend to have a higher salary range on average.
# Entry-level jobs offer significantly lower pay.

# Recommendation:
# Focus job scraping on higher-level positions to target premium salary ranges.

# Prediction:
# As remote work trends grow, mid-level roles may see upward salary adjustment.


# ======================================================
# 🕵️ Remote Jobs - Diagnostic Query
# ======================================================

# Business Question:
# Which companies are posting the most high-paying jobs?

sql_query = '''
WITH high_salary_jobs AS (
    SELECT 
        company_name,
        annual_salary_max,
        ROW_NUMBER() OVER (PARTITION BY company_name ORDER BY annual_salary_max DESC) AS rank
    FROM remote_jobs
    WHERE annual_salary_max > 100000
)
SELECT 
    company_name,
    COUNT(*) AS high_paying_jobs
FROM high_salary_jobs
WHERE rank <= 5
GROUP BY company_name
ORDER BY high_paying_jobs DESC;
'''

remote_highpay_df = pd.read_sql(sql_query, con=engine)


# Insight:
# Certain companies consistently post high-paying jobs above 100k.

# Recommendation:
# Consider building partnerships with those companies or tracking their listings.

# Prediction:
# Their hiring trends could be indicators of high growth sectors.


In [10]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import seaborn as sns

# Create a PDF file to store all figures
with PdfPages("Visualizations.pdf") as pdf:
    # Plot 1: Remote Jobs - Salary by Level
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=remote_salary_df,
        x="avg_salary_max",
        y="job_level",
        palette="viridis"
    )
    plt.title("Average Max Salary by Job Level (Remote Jobs)")
    plt.xlabel("Average Max Salary ($)")
    plt.ylabel("Job Level")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    # Plot 2: Remote Jobs - High Paying by Company
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=remote_highpay_df.sort_values("high_paying_jobs", ascending=False),
        x="high_paying_jobs",
        y="company_name",
        palette="rocket"
    )
    plt.title("High-Paying Job Postings by Company (>$100k)")
    plt.xlabel("Number of High-Paying Jobs")
    plt.ylabel("Company Name")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    # Plot 3: Indeed - Most Common Job Titles
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=indeed_jobs_summary_df,
        x="job_count",
        y="job_title",
        palette="coolwarm"
    )
    plt.title("Top 10 Job Titles on Indeed")
    plt.xlabel("Job Count")
    plt.ylabel("Job Title")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    # Plot 4: Indeed - Top Companies by Job Title
    plt.figure(figsize=(12, 8))
    sns.barplot(
        data=indeed_top_companies_df.sort_values("count", ascending=False),
        x="count",
        y="job_title",
        hue="company_name",
        dodge=False
    )
    plt.title("Top Companies for Most Common Job Titles")
    plt.xlabel("Job Post Count")
    plt.ylabel("Job Title")
    plt.legend(title="Company", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    pdf.savefig()
    plt.close()



/var/folders/s4/h0k4q7kx7s52yzr_bd9cvj140000gn/T/ipykernel_19095/938394005.py:9: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/var/folders/s4/h0k4q7kx7s52yzr_bd9cvj140000gn/T/ipykernel_19095/938394005.py:24: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(
/var/folders/s4/h0k4q7kx7s52yzr_bd9cvj140000gn/T/ipykernel_19095/938394005.py:39: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


## Explanation of how visualizations were created 
s
The visualizations in the Visualizations.pdf file were created using Python by first querying the PostgreSQL database with pandas.read_sql() to create four dataframes based on descriptive and diagnostic analytics queries for both the remote_jobs and indeed_jobs datasets. Each dataframe was then visualized using matplotlib and seaborn to generate bar charts and other relevant plots that illustrate the insights and trends identified in the SQL queries. All visualizations were compiled into a single multi-page PDF file using matplotlib.backends.backend_pdf.PdfPages, resulting in a consolidated report that can be easily shared or reviewed.